<a href="https://colab.research.google.com/github/nguyenvudev20/mse22.Coursera_ML_AI/blob/main/AssignmentEnd3.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Prices Competition for Kaggle 



In [1]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import numpy as np




# tạo các tham số môi trường cho api kaggle để kết nối tải file 
os.environ['KAGGLE_USERNAME'] = "vunguyen2020"
os.environ['KAGGLE_KEY'] = "ceaa092a89951b4d01485d4ee50cf382"

In [3]:
!kaggle competitions download -c home-data-for-ml-course #tải gói data cho bài tutorial từ kaggle dưới dạng zip file
!mkdir kaggle #tạo folder để unzip file vừa tải
!unzip /content/home-data-for-ml-course.zip -d kaggle #tiến hành unzip vào folder kaggle

  0% 0.00/386k [00:00<?, ?B/s]
100% 386k/386k [00:00<00:00, 63.8MB/s]
Archive:  /content/home-data-for-ml-course.zip
  inflating: kaggle/data_description.txt  
  inflating: kaggle/sample_submission.csv  
  inflating: kaggle/sample_submission.csv.gz  
  inflating: kaggle/test.csv         
  inflating: kaggle/test.csv.gz      
  inflating: kaggle/train.csv        
  inflating: kaggle/train.csv.gz     


In [4]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv("kaggle/train.csv")
# Hiển thị 5 dòng đầu tiên của dữ liệu
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# xem thucộ tính của các trường dữ liệu
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [6]:
# dữ liệu có 1460 dòng và 81 trường dữ liệu
train_data.shape

(1460, 81)

In [7]:
# lựa chọn các trường dữ liệu
train_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [12]:
features = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr',
    'KitchenAbvGr', 'TotRmsAbvGrd', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
X= train_data[features]
y=train_data['SalePrice']

In [13]:
X.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,...,TotRmsAbvGrd,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,856,854,0,1710,...,8,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,1262,0,0,1262,...,6,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,920,866,0,1786,...,6,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,961,756,0,1717,...,7,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,1145,1053,0,2198,...,9,192,84,0,0,0,0,0,12,2008


In [14]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

## Thực hiện trainning dựa trên model của Tensorflow Keras

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu đầu vào
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
import tensorflow.keras.backend as K

# tạo hàm r_squared để đánh giá model
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Khỏi tạo model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), RootMeanSquaredError(), MeanAbsoluteError(), r_squared])

# Huấn luyện model trên tập huấn luyện
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50
37/37 [==============================] - 1s 12ms/step - loss: 38884986880.0000 - mean_squared_error: 38884986880.0000 - root_mean_squared_error: 197192.7656 - mean_absolute_error: 181439.9375 - r_squared: -6.4704 - val_loss: 39652749312.0000 - val_mean_squared_error: 39652749312.0000 - val_root_mean_squared_error: 199129.9844 - val_mean_absolute_error: 178836.4531 - val_r_squared: -5.7364
Epoch 2/50
37/37 [==============================] - 0s 4ms/step - loss: 38883028992.0000 - mean_squared_error: 38883028992.0000 - root_mean_squared_error: 197187.7969 - mean_absolute_error: 181434.9375 - r_squared: -6.5378 - val_loss: 39649595392.0000 - val_mean_squared_error: 39649595392.0000 - val_root_mean_squared_error: 199122.0625 - val_mean_absolute_error: 178828.5938 - val_r_squared: -5.7358
Epoch 3/50
37/37 [==============================] - 0s 4ms/step - loss: 38877745152.0000 - mean_squared_error: 38877745152.0000 - root_mean_squared_error: 197174.4062 - mean_absolute_error: 18142

In [30]:
# chạy thử mô hình với tập kiểm tra
model.evaluate(X_test, y_test)

# Đánh giá hiệu suất mô hình trên tập kiểm tra
print("Mean Squared Error:", model.metrics[0].result().numpy())
print("Root Mean Squared Error:", model.metrics[1].result().numpy())
print("Mean Absolute Error:", model.metrics[2].result().numpy())
print("R-squared:", model.metrics[3].result().numpy())


10/10 [==============================] - 0s 3ms/step - loss: 8465224192.0000 - mean_squared_error: 8465224192.0000 - root_mean_squared_error: 92006.6562 - mean_absolute_error: 79019.3984 - r_squared: -0.5141
Mean Squared Error: 8465224000.0
Root Mean Squared Error: 8465224000.0
Mean Absolute Error: 92006.66
R-squared: 79019.4


### Kết luận

sau khi thực hiện train model và ta được kết quả:
- Mean Squared Error: 8465224000.0
- Root Mean Squared Error: 8465224000.0
- Mean Absolute Error: 92006.66
- R-squared: 79019.4


## Sử dụng Hyperparameter Tuning để tối ưu model